###### author: Barzan saeedpour
# Image retrieval
### using a denoising auto encoder for feature extraction

### objective: give a test image to the algorithm and take similar images back (from training data).

<img src="image_retrival.png">

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.datasets import mnist
import numpy as np
from matplotlib import pyplot as plt

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

np.save('x_train',x_train)

noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)


def train_model():
    input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same', name='encoder')(x)

    # at this point the representation is (4, 4, 8) i.e. 128-dimensional

    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

    autoencoder.fit(x_train_noisy, x_train,
                    epochs=20,
                    batch_size=128,
                    shuffle=True,
                    validation_data=(x_test_noisy, x_test),
                    callbacks=[TensorBoard(log_dir='/tmp/tb', histogram_freq=0, write_graph=False)])

    autoencoder.save('denoising_autoencoder.h5')

train_model()


In [ ]:
image_index = 6 # a specific image use for test the results

In [ ]:
import numpy as np
from keras.models import Model
from keras.datasets import mnist
import cv2
from keras.models import load_model
from sklearn.metrics import label_ranking_average_precision_score
import time


print('Loading model :')
t0 = time.time()
autoencoder = load_model('denoising_autoencoder.h5')
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('encoder').output)
t1 = time.time()
print('Model loaded in: ', t1-t0)



In [ ]:
def plot_denoised_images():
    denoised_images = autoencoder.predict(x_test_noisy.reshape(x_test_noisy.shape[0], x_test_noisy.shape[1], x_test_noisy.shape[2], 1))
    test_img = x_test_noisy[image_index]
    resized_test_img = cv2.resize(test_img, (280, 280))
    fig2, ax2 = plt.subplots(nrows=1, ncols=2)
    ax2[0].imshow(resized_test_img.reshape(280,280),cmap='gray')
    ax2[0].set_xlabel('noisy image (input)', fontsize=13)

    output = denoised_images[image_index]
    resized_output = cv2.resize(output, (280, 280))
    ax2[1].imshow(resized_output.reshape(280,280),cmap='gray')
    ax2[1].set_xlabel('denoised image (output)', fontsize=13)
    
# To plot a denoised image
plot_denoised_images()

In [ ]:
a = np.array([1, 2, 3])
b = np.array([2, 3, 4])
np.stack((a, b), axis=-1)

In [ ]:
x = np.array([3, 1, 2])
np.argsort(x)

In [ ]:
def retrieve_closest_images(test_element, n_samples=10):
    learned_codes = encoder.predict(x_train)
    learned_codes = learned_codes.reshape(learned_codes.shape[0],
                                          learned_codes.shape[1] * learned_codes.shape[2] * learned_codes.shape[3])

    test_code = encoder.predict(np.array([test_element]))
    test_code = test_code.reshape(test_code.shape[1] * test_code.shape[2] * test_code.shape[3])

    distances = []

    for code in learned_codes:
        distance = np.linalg.norm(code - test_code)
        distances.append(distance)
    nb_elements = learned_codes.shape[0]
    distances = np.array(distances)
    learned_code_index = np.arange(nb_elements)

    distance_with_index = np.stack((distances, learned_code_index), axis=-1)

    sorted_distances_with_index = distance_with_index[distances.argsort()]

    sorted_indexes = sorted_distances_with_index[:, 1]
    kept_indexes = sorted_indexes[:n_samples]

    original_image = x_test[image_index]
    fig1, ax1 = plt.subplots()
    ax1.imshow(original_image.reshape(28,28),cmap='gray')
    ax1.set_xlabel('input', fontsize=13)


    fig2, ax2 = plt.subplots(nrows=1,ncols=n_samples,)
    for i in range(0, n_samples):
        ax2[i].imshow(x_train[int(kept_indexes[i]), :].reshape(x_train[0].shape[0],x_train[0].shape[1]),cmap='gray')
   
    ax2[0].set_xlabel('retrieved images', fontsize=13)


In [ ]:
image_index = 4449 # a specific image use for test the results

# To retrieve closest image
retrieve_closest_images(x_test[image_index],10)